# **■ 出向人件費の金額突合用資料作成**
一部データを削除しております。


【更新履歴】

*   ソースデータとして法人取引先別元帳を追加
*   出向関係の組合せによるフィルタリングは行わない
*   組合せ確認用資料として、人件費エクセルとソースデータの組合せを出力（お互い組合せが抜けてるケースアリ）
*   略名称・会社コード・取引先コードの参照リストを人件費エクセルとグループ会社一覧から動的生成
*   勘定科目が出向人件費（受入出向人件費）で、取引先が関係会社以外を除外



**(1) データソースにBiz出力の「Tableau用データ」を使う場合**

※Tableau用の最新JOURNAL_SLIPデータを「会計データ」フォルダに入れるのを忘れないように‼‼‼

In [ ]:
import os
import pandas as pd # 行列データを操作するライブラリ
import shutil # ファイル操作するライブラリ
import datetime # 日付・時間を操作するライブラリ

from google.colab import drive # Google Driveを操作するライブラリ
drive.mount('/content/drive')

folder_path = "/
pd.set_option('display.max_rows', 10)
import warnings
warnings.simplefilter('ignore')

Mounted at /content/drive


In [ ]:
# 会計年月を入力
yyyymm = input("会計年月を入力してください(例 2024年1月→202401): ") # 会計年月の入力(テキスト型)



# JOURNAL_SLIPフォルダのルートパス
slip_path = os.path.join("/会計データ/JOURNAL_SLIP", yyyymm)


# 取引明細データjournal_dtlと取引ヘッダデータjournal_headerの取得
# 明細及びヘッダデータのそれぞれの{列名:型}の辞書定義
header_dtype = {'仕訳ID':'int', '会社コード':'str', '仕訳種別コード':'int', '仕訳状態コード':'int', '仕訳種類コード':'int', '承認区分':'int', '訂正区分':'int', '計上日':'str', '実施番号':'str', '照合キー':'str',
                '作成アプリケーション':'str', '作成アプリケーション分類':'str', '伝票摘要':'str', '登録組織コード':'str', '更新組織コード':'str', '登録者名':'str', '登録日':'str', '更新者名':'str', '更新日':'str', '仕訳番号':'int', '決算種別コード':'int'}
header_columns = list(header_dtype.keys())

dtl_dtype = {'仕訳ID':'int', '会社コード':'str', '貸借区分':'int', '行番号':'int', '消込枝番号':'int', '計上組織コード':'str', '取引先コード':'str', '社員コード':'str', '勘定科目コード':'int', '補助科目コード':'str',
             '入力金額':'int', '入力税額':'int', '税処理区分':'int', '消費税コード':'str', '明細摘要':'str', '分析フィールド3':'str', '分析フィールド4':'str', '分析テキスト11':'str', '相手勘定科目コード':'int', '相手補助科目コード':'str'}
dtl_columns = list(dtl_dtype.keys())



journal_header = pd.read_csv(slip_path + "/JOURNAL_HEADER.txt", sep=" ", header=None, names= header_columns, dtype = header_dtype)
journal_dtl = pd.read_csv(slip_path + "/JOURNAL_DTL.txt", sep=" ", header=None, names= dtl_columns, dtype = dtl_dtype)



#################################################
#  各マスタデータの取得（要素0はファイル名、要素1は{列名:型}の辞書、要素2はマージ時のキーリスト）
master_list = [
    ["/ACCOUNT_CMN_MASTER.txt", {'会社コード':'str', '勘定科目コード':'int', '勘定科目名':'str'}, ["会社コード", "勘定科目コード"]], #勘定科目マスタ
    ["/ACCNT_CNST_EXPN_FS_TYPE_MASTER.txt", {'会社コード':'str', '勘定科目コード':'int', '財務諸表区分':'int'}, ["会社コード", "勘定科目コード"]], #勘定科目別の財務諸表区分マスタ
    ["/ACCOUNT_BALANCE_TYPE_MASTER.txt", {'会社コード':'str', '勘定科目コード':'int', '貸借区分':'int'}, ["会社コード", "勘定科目コード"]], #勘定科目別の貸借区分マスタ
    ["/SUBSIDIARY_ACCNT_CMN_MASTER.txt", {'会社コード':'str', '勘定科目コード':'int', '補助科目コード':'str', '補助科目名':'str'}, ["会社コード", "勘定科目コード", "補助科目コード"]], #勘定科目別の補助科目マスタ
    ["/CSTMR_CMN_MASTER.txt", {'会社コード':'str', '取引先コード':'str', '取引先名':'str'}, ["会社コード", "取引先コード"]], #取引先マスタ
    ["/INDIRECT_TAX_CMN_MASTER.txt", {'会社コード':'str', '消費税コード':'str', '税名称':'str', 'tmp_1':'int', 'tmp_2':'float', 'tmp_3':'float'}, ["会社コード", "消費税コード"]] #消費税マスタ
]


for i in range(len(master_list)):
    master = pd.read_csv(slip_path + master_list[i][0], sep=" ", header=None, names=list(master_list[i][1].keys()), dtype = master_list[i][1])
    journal_dtl = pd.merge(journal_dtl, master, on=master_list[i][2], how="left")


# マージしたマスタデータとジャーナルデータをマージ
join_data = pd.merge(journal_dtl, journal_header, on="仕訳ID", how="left")



# 会社コード「2016A」のせいで今後会社コードは全てstr型（データフレーム検索時など要注意）
#join_data["会社コード_x"] =join_data["会社コード_x"].astype(int)
#join_data["会社コード_y"] =join_data["会社コード_y"].astype(int)



# データ項目の整理（第一次）
join_data = join_data[['仕訳ID', '会社コード_x', '貸借区分_x', '計上組織コード', '取引先コード',
       '勘定科目コード', '補助科目コード', '入力金額', '入力税額', '税処理区分', '消費税コード',
       '明細摘要', '相手勘定科目コード', '相手補助科目コード',
       '勘定科目名', '貸借区分_y', '補助科目名', '取引先名', '会社コード_y', '仕訳種別コード','承認区分', '訂正区分',
       '計上日', '伝票摘要', '登録者名', '登録日', '更新者名', '更新日', '仕訳番号', ]]


# 再申請待ち、申請中止の仕訳を除外
join_data = join_data[join_data["承認区分"] != 0]
join_data = join_data[join_data["承認区分"] != 3]


# 訂正区分で黒伝票のみ抽出（赤・元黒除外）
join_data = join_data[join_data["訂正区分"] == 0]


# 伝票摘要内の「0001」などの枝番（仕訳件数が1000件超の場合に対応するため枝番で分割してる）を削除
slip_list = join_data["伝票摘要"].str.replace('_0001', '').str.replace('_0002', '').str.replace('_0003', '').str.replace('_0004', '')
join_data["伝票摘要"] = slip_list

会計年月を入力してください(例 2024年1月→202401): 202407


**(2) データソースにBiz出力の「法人取引先別元帳」を使う場合**

In [ ]:
import pandas as pd
import glob

from google.colab import drive # Google Driveを操作するライブラリ
drive.mount('/content/drive')


folder_path = "/
pd.set_option('display.max_rows', 10)
import warnings
warnings.simplefilter('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 会計年月を入力
yyyymm = input("会計年月を入力してください(例 2024年1月→202401): ") # 会計年月の入力(テキスト型)


source_paths = glob.glob(folder_path + "/法人取引先別元帳ソース/{yyyymm}/*法人取引先別元帳*.csv")
print(f"ソースとなる法人取引先別元帳の数（カレント数）は{len(source_paths)}")


# 各カレントの法人取引先別元帳の読込・合成
df_journal = pd.DataFrame()

for source_path in source_paths:
    df = pd.read_csv(source_path, encoding = "utf-8", dtype = {"会社コード":"str", "補助科目コード":"str"}, low_memory=False)
    df_journal = pd.concat([df_journal, df])


# 法人取引先別元帳をTableauのjoin_dataと同じにするために列名の変更や列追加、その他整形
df_journal.rename(columns={"会社コード": "会社コード_x", "貸借区分": "貸借区分_x"}, inplace=True)

df_journal["借方入力金額"] = df_journal["借方入力金額"].fillna(0)
df_journal["貸方入力金額"] = df_journal["貸方入力金額"].fillna(0)
df_journal["借方入力税額"] = df_journal["借方入力税額"].fillna(0)
df_journal["貸方入力税額"] = df_journal["貸方入力税額"].fillna(0)
df_journal["入力金額"] = df_journal["借方入力金額"] + df_journal["貸方入力金額"]
df_journal["入力税額"] = df_journal["借方入力税額"] + df_journal["貸方入力税額"]

df_journal["会社コード_y"] = ""
df_journal["貸借区分_y"] = ""
df_journal["仕訳種別コード"] = ""
df_journal["仕訳ID"] = ""



# データ項目の整理（第一次）
join_data = df_journal[['仕訳ID', '会社コード_x', '貸借区分_x', '計上組織コード', '取引先コード',
       '勘定科目コード', '補助科目コード', '入力金額', '入力税額', '税処理区分', '消費税コード',
       '明細摘要', '相手勘定科目コード', '相手補助科目コード',
       '勘定科目名', '貸借区分_y', '補助科目名', '取引先名', '会社コード_y', '仕訳種別コード','承認区分', '訂正区分',
       '計上日', '伝票摘要', '登録者名', '登録日', '更新者名', '更新日', '仕訳番号', ]]



# 再申請待ち、申請中止の仕訳を除外
join_data = join_data[join_data["承認区分"] != 0]
join_data = join_data[join_data["承認区分"] != 3]


# 訂正区分で黒伝票のみ抽出（赤・元黒除外）
join_data = join_data[join_data["訂正区分"] == 0]


# 仕訳番号の整形（14桁から5桁（※非0パディング）に変更）
join_data["仕訳番号"] = join_data["仕訳番号"].map(lambda x: int(x[-5:]))



# 伝票摘要内の「0001」などの枝番（仕訳件数が1000件超の場合に対応するため枝番で分割してる）を削除
# なぜか法人取引先別元帳では伝票摘要の枝番の頭のアンダーバーが消えてる
slip_list = join_data["伝票摘要"].str.replace(' 0001', '').str.replace(' 0002', '').str.replace(' 0003', '').str.replace(' 0004', '')
join_data["伝票摘要"] = slip_list

会計年月を入力してください(例 2024年1月→202401): 202406
ソースとなる法人取引先別元帳の数（カレント数）は10


**・会社略名称とBiz会社コード・取引先コードの参照マスタを動的作成**

In [ ]:
# グループ会社一覧のエクセルファイル名　※最新のファイル名か確認‼‼
group_list_name = "A9_グループ会社一覧(45期-2024年6月).xlsx"

In [ ]:
# 人件費エクセルの企業一覧から略名称とBiz会社コード、COMPANY会社コードを取得
excel_path = f"/content/drive/Shareddrives/01_組織_TSS_財務経理本部/★人件費関連/{yyyymm}/人件費{yyyymm}.xlsx"
df_master1 =  pd.read_excel(excel_path, sheet_name = "企業一覧", dtype={"会社コード(Biz)" : str, "企業CD" : str})[["略名称", "会社コード(Biz)", "企業CD"]]
df_master1 = df_master1.dropna(subset=["企業CD"])

# グループ会社一覧から取引先コードとBiz会社コード、COMPANY会社コードを取得
excel_path2 = "/content/drive/Shareddrives/01_組織_TSS_財務経理本部/子会社経理資料/セグメント会計/"
df_master2 =  pd.read_excel(excel_path2 + group_list_name, sheet_name = "関連グループ企業マスタ", dtype={"会社コード(COMPANY)" : str})[["取引先コード", "会社コード(COMPANY)"]]
df_master2 = df_master2.dropna(subset=["会社コード(COMPANY)"])


# 上記2つをCOMPANY会社コードでマージ（※Biz会社コード無し取引先コード有りのグループ会社があるのでBiz会社コードはキーにできない）
df_master = pd.merge(df_master1, df_master2, left_on = "企業CD", right_on = "会社コード(COMPANY)", how = "outer").drop(columns="会社コード(COMPANY)")
df_master = df_master.dropna(subset=["略名称"]).dropna(subset=["取引先コード"])
df_cmpny_num = df_master[["略名称", "会社コード(Biz)", "取引先コード"]].rename(columns={"会社コード(Biz)" : "会社コード"})

In [ ]:
df_cmpny_num


,略名称,会社コード,取引先コード
0,THD,0001,T050020028
1,TRC,1001,T050020031
4,TRS,1006,T050020029
5,MLS,2006,T050020003
6,TRE-C,NaN,T050020000
...,...,...,...
35,ATR,1010,T050023890
36,SP,NaN,T020022744
37,TGRA,NaN,T050024578
38,TGRO,NaN,T050024579


**・当月の出向関係の組合せリストを動的作成（ソースデータと人件費エクセルのマージ）**

In [ ]:
# 当月人件費エクセルの「出向給与集計」と「当月手当集計(出向)」シートの読込、結合
excel_path = f"/content/drive/Shareddrives/01_組織_TSS_財務経理本部/★人件費関連/{yyyymm}/人件費{yyyymm}.xlsx"

df1 = pd.read_excel(excel_path, sheet_name = "出向給与集計")[["出向元略", "出向先略"]]
df2 = pd.read_excel(excel_path, sheet_name = "当月手当集計(出向)")[["出向元略", "出向先略"]]

_df_excel = pd.concat([df1, df2], axis=0, ignore_index=True)


# 空の行と重複行の削除
df_excel = _df_excel.dropna().drop_duplicates()

In [ ]:
# 勘定科目833400と833100のそれぞれについてjoin_dataを参照し出向関係の組合せを取得
cond_check1 = (join_data["勘定科目コード"] == 833400)
df_check1 = join_data[cond_check1][["会社コード_x", "取引先コード"]]

# 出向相手が関係会社以外の仕訳を除外
# ※システム上、取引先コード「T05」をキーにしているがT05以外のコードの関係会社もありうる点は留意
cond_t05 = df_check1["取引先コード"].str.contains("T05")
df_check1 = df_check1[cond_t05]


df_check1['出向元略'] = df_check1["会社コード_x"].apply(lambda cmpny_code : df_cmpny_num[df_cmpny_num["会社コード"] == cmpny_code]["略名称"].iloc[-1])
df_check1['出向先略'] = df_check1["取引先コード"].apply(lambda vender_code : df_cmpny_num[df_cmpny_num["取引先コード"] == vender_code]["略名称"].iloc[-1])



cond_check2 = (join_data["勘定科目コード"] == 833100)
df_check2 = join_data[cond_check2][["会社コード_x", "取引先コード"]]

cond_t05 = df_check2["取引先コード"].str.contains("T05")
df_check2 = df_check2[cond_t05]


df_check2['出向先略'] = df_check2["会社コード_x"].apply(lambda cmpny_code : df_cmpny_num[df_cmpny_num["会社コード"] == cmpny_code]["略名称"].iloc[-1])
df_check2['出向元略'] = df_check2["取引先コード"].apply(lambda vender_code : df_cmpny_num[df_cmpny_num["取引先コード"] == vender_code]["略名称"].iloc[-1])


_df_source = pd.concat([df_check1, df_check2], axis=0, ignore_index=True)


# 空の行と重複行の削除
df_source = _df_source[["出向元略", "出向先略"]].dropna().drop_duplicates()


# 当月の出向関係組合せリストの作成
df_couple = pd.concat([df_excel, df_source], axis=0).drop_duplicates().reset_index(drop=True)
#df_couple.to_csv(folder_path+f'丸山作業フォルダ/出向人件費金額突合/Output/出向組合せ一覧{yyyymm}.csv', index=False)

In [ ]:
df_couple

,出向元略,出向先略
0,THD,TRC
1,THD,RAI
2,TRC,THD
3,TRC,TRS
4,TRC,ATR
...,...,...
29,TRS,MJY
30,RAI,THD
31,RAI,TRC
32,RAI,SPY


In [ ]:
# df_excel(=left)またはdf_check(=right)にしかない組合せの確認
# df_source に変更（df_checkから）
pd.merge(df_excel,df_source, on=["出向元略", "出向先略"], how ="outer", indicator=True).query(f'_merge != "both"')

,出向元略,出向先略,_merge
13,TRC,SP,left_only
23,TRE,TTC,left_only
24,MJY,TRS,right_only
25,TRC,neri,right_only
26,MLS,TRS,right_only
...,...,...,...
28,TRS,MJY,right_only
29,RAI,THD,right_only
30,RAI,TRC,right_only
31,RAI,SPY,right_only


In [ ]:
# 念のため関係会社以外として仕訳を除外した出向相手の取引先コードを確認
check_list = []


df_833400 = join_data[join_data["勘定科目コード"] == 833400]

for cmpny_num, vender_num in zip(df_833400["会社コード_x"], df_833400["取引先コード"]) :
    if vender_num in list(df_cmpny_num["取引先コード"]):
        continue
    else:
       check_list.append((cmpny_num, vender_num))


df_833100 = join_data[join_data["勘定科目コード"] == 833100]

for cmpny_num, vender_num in zip(df_833100["会社コード_x"], df_833100["取引先コード"]) :
    if vender_num in list(df_cmpny_num["取引先コード"]):
        continue
    else:
        check_list.append((cmpny_num, vender_num))


set(check_list)

{('0001', 'T060060177'),
 ('0001', 'T060060182'),
 ('1001', 'T020019161'),
 ('1009', '_'),
 ('1010', '_'),
 ('2006', 'T020021538')}

**・出向元の金額リスト作成**

In [ ]:
  #貸借区分が0(借方)の入力金額、入力税額をマイナスにする関数
def div_check_amount(df):
    if df["貸借区分_x"] == 0:
      return -df['入力金額']
    elif df["貸借区分_x"] == 1:
      return df['入力金額']
    else:
      return str(df["入力金額"])+"※貸借区分不明"


def div_check_tax(df):
    if df["貸借区分_x"] == 0:
      return -df['入力税額']
    elif df["貸借区分_x"] == 1:
      return df['入力税額']
    else:
      return str(df["入力税額"])+"※貸借区分不明"

In [ ]:
# まとめてCSVで出力（後でExcelに読み込ませて、出向元/出向先でフィルターをかけて使用）


#① 勘定科目「受入出向人件費」の抽出
cond_833400 = join_data["勘定科目コード"] == 833400
df = join_data[cond_833400]


# 出向相手が関係会社以外の仕訳を除外
cond_t05 = df["取引先コード"].str.contains("T05")
df = df[cond_t05]


#② データ内の仕訳番号は下５桁以下の数字（０パディングなし）なので正式なものに変換
df["仕訳番号_DTL"] = df["仕訳番号"].apply(lambda num: str(yyyymm) + "000000000"+ str(num).zfill(5))



#③ 貸借区分が0(借方)の入力金額、入力税額をマイナスにする
df['貸借金額'] = df.apply(div_check_amount, axis=1)
df['貸借税額'] = df.apply(div_check_tax, axis=1)



#④ 出向元と取引先の略名称の新規列追加
df['出向元'] = df["会社コード_x"].apply(lambda cmpny_code : df_cmpny_num[df_cmpny_num["会社コード"] == cmpny_code]["略名称"].iloc[-1])
df['取引先'] = df["取引先コード"].apply(lambda vender_code : df_cmpny_num[df_cmpny_num["取引先コード"] == vender_code]["略名称"].iloc[-1])



#⑤ Nullの勘定科目名を埋める
df["勘定科目名"] = df["勘定科目名"].fillna("受入出向人件費")


#⑥ データ項目の整理（第二次）
df = df[['出向元', '取引先', '仕訳ID', '会社コード_x', '計上組織コード', '計上日', '取引先コード', '取引先名', '貸借区分_x', '貸借区分_y',
      '勘定科目コード', '勘定科目名', '補助科目コード', '補助科目名', '伝票摘要', '明細摘要',
      '貸借金額', '貸借税額', '税処理区分', '消費税コード','登録者名', '仕訳番号_DTL']]



# CSVで保存
df.to_csv(folder_path+f'丸山作業フォルダ/出向人件費金額突合/Output/≪出向元明細≫{yyyymm}ソース.csv', index=False)

**・出向先の金額リストの作成**

In [ ]:
#貸借区分が1(貸方)の入力金額、入力税額をマイナスにする関数
def div_check_amount2(df2):
    if df2["貸借区分_x"] == 0:
      return df2['入力金額']
    elif df2["貸借区分_x"] == 1:
      return -df2['入力金額']
    else:
      return str(df2["入力金額"])+"※貸借区分不明"


def div_check_tax2(df2):
    if df2["貸借区分_x"] == 0:
      return df2['入力税額']
    elif df2["貸借区分_x"] == 1:
      return -df2['入力税額']
    else:
      return str(df2["入力税額"])+"※貸借区分不明"

In [ ]:
# まとめてCSVで出力（後でExcelに読み込ませて、出向元/出向先でフィルターをかけて使用）


#① 勘定科目「出向人件費」の抽出
cond_833100 = join_data["勘定科目コード"] == 833100
df2 = join_data[cond_833100]


# 出向相手が関係会社以外の仕訳を除外
cond_t05 = df2["取引先コード"].str.contains("T05")
df2 = df2[cond_t05]



#② データ内の仕訳番号は下５桁以下の数字（０パディングなし）なので正式なものに変換
df2["仕訳番号_DTL"] = df2["仕訳番号"].apply(lambda num: str(yyyymm) + "000000000"+ str(num).zfill(5))



#③ 貸借区分が0(借方)の入力金額、入力税額をマイナスにする
df2['貸借金額'] = df2.apply(div_check_amount2, axis=1)
df2['貸借税額'] = df2.apply(div_check_tax2, axis=1)



#④ 出向元と取引先の略名称の新規列追加
df2['出向先'] = df2["会社コード_x"].apply(lambda cmpny_code : df_cmpny_num[df_cmpny_num["会社コード"] == cmpny_code]["略名称"].iloc[-1])
df2['取引先'] = df2["取引先コード"].apply(lambda vender_code : df_cmpny_num[df_cmpny_num["取引先コード"] == vender_code]["略名称"].iloc[-1])



#⑤ Nullの勘定科目名を埋める
df2["勘定科目名"] = df2["勘定科目名"].fillna("出向人件費")


#⑥ データ項目の整理（第二次）
df2 = df2[['出向先', '取引先', '仕訳ID', '会社コード_x', '計上組織コード', '計上日', '取引先コード', '取引先名', '貸借区分_x', '貸借区分_y',
        '勘定科目コード', '勘定科目名', '補助科目コード', '補助科目名', '伝票摘要', '明細摘要',
        '貸借金額', '貸借税額', '税処理区分', '消費税コード','登録者名', '仕訳番号_DTL']]



# CSVで保存
df2.to_csv